**PCB ERROR DETECTION AND CLASSIFICATION**

Data Preparation and Image Processing

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
rgb_template_img = cv2.imread('/content/drive/MyDrive/PCB_dataset/PCB_DATASET/PCB_USED/01.JPG')
a = int(rgb_template_img.shape[0]/4)
b = int(rgb_template_img.shape[1]/4)

template_img = cv2.imread('/content/drive/MyDrive/PCB_dataset/PCB_DATASET/PCB_USED/01.JPG', 0)
template_img_resize = cv2.resize(template_img, (b,a))
blur_template_img = cv2.GaussianBlur(template_img_resize, (3,3),0)

template_adap_thresh = cv2.adaptiveThreshold(
    blur_template_img, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY, 15, 5
)
folder_path = "/content/drive/MyDrive/PCB_dataset/PCB_DATASET/images/01_Spur/"
image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(".jpg")])
image_files


['01_spur_01.jpg',
 '01_spur_02.jpg',
 '01_spur_03.jpg',
 '01_spur_04.jpg',
 '01_spur_05.jpg',
 '01_spur_06.jpg',
 '01_spur_07.jpg',
 '01_spur_08.jpg',
 '01_spur_09.jpg',
 '01_spur_10.jpg',
 '01_spur_11.jpg',
 '01_spur_12.jpg',
 '01_spur_13.jpg',
 '01_spur_14.jpg',
 '01_spur_15.jpg',
 '01_spur_16.jpg',
 '01_spur_17.jpg',
 '01_spur_18.jpg',
 '01_spur_19.jpg',
 '01_spur_20.jpg']

In [ ]:
patch_id = 0
for file_name in image_files:

    img_path = os.path.join(folder_path, file_name)
    print("Processing:", file_name)

    rgb_test_img = cv2.imread(img_path)
    test_img = cv2.imread(img_path, 0)

    test_img_resize = cv2.resize(test_img, (b,a))
    blur_test_img = cv2.GaussianBlur(test_img_resize, (3,3),0)

    test_adap_thresh = cv2.adaptiveThreshold(
        blur_test_img, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY, 15, 5
    )

    sub_img = cv2.subtract(template_adap_thresh, test_adap_thresh)
    final_img = cv2.medianBlur(sub_img, 3)

    cnts = cv2.findContours(final_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]
    blobs = [cnt for cnt in cnts if 0 < cv2.contourArea(cnt) < 300]

    print("Number of defects:", len(blobs))

    orig = test_img_resize
    mask_img = final_img
    out_dir = "/content/drive/MyDrive/PCB_dataset/PCB_DATASET/Defects/Spur"
    os.makedirs(out_dir, exist_ok=True)

    _, thresh = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    h_img, w_img = mask_img.shape[:2]


    for cnt in contours:
        if 0 < cv2.contourArea(cnt) < 300:

            x, y, w, h = cv2.boundingRect(cnt)
            cx = x + w // 2
            cy = y + h // 2

            bw, bh = 64, 64
            x0 = max(0, cx - bw // 2)
            y0 = max(0, cy - bh // 2)
            x1 = min(w_img, x0 + bw)
            y1 = min(h_img, y0 + bh)

            patch = orig[y0:y1, x0:x1]

            if patch.shape != (bh, bw):
                continue

            out_path = os.path.join(out_dir, f"defect_{patch_id:04d}.png")
            cv2.imwrite(out_path, patch)
            patch_id += 1


Processing: 01_spur_01.jpg
Number of defects: 2
Processing: 01_spur_02.jpg
Number of defects: 2
Processing: 01_spur_03.jpg
Number of defects: 3
Processing: 01_spur_04.jpg
Number of defects: 2
Processing: 01_spur_05.jpg
Number of defects: 3
Processing: 01_spur_06.jpg
Number of defects: 2
Processing: 01_spur_07.jpg
Number of defects: 2
Processing: 01_spur_08.jpg
Number of defects: 2
Processing: 01_spur_09.jpg
Number of defects: 2
Processing: 01_spur_10.jpg
Number of defects: 2
Processing: 01_spur_11.jpg
Number of defects: 2
Processing: 01_spur_12.jpg
Number of defects: 2
Processing: 01_spur_13.jpg
Number of defects: 2
Processing: 01_spur_14.jpg
Number of defects: 2
Processing: 01_spur_15.jpg
Number of defects: 2
Processing: 01_spur_16.jpg
Number of defects: 2
Processing: 01_spur_17.jpg
Number of defects: 2
Processing: 01_spur_18.jpg
Number of defects: 2
Processing: 01_spur_19.jpg
Number of defects: 2
Processing: 01_spur_20.jpg
Number of defects: 2
